In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa
import seaborn as sns

In [ ]:
EXPORT = 100
CO2L = 0.90
CLUSTERS = 4
H = 3
LL = "3.0"
DISCOUNT_RATE = 0.15

RUN = "dyntrans_full_3H_ws"
CATEGORY = "postnetworks"
SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{CO2L}0_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

#OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
OUTPUT = PREFIX + "results"
INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/graphics/demand/{SCENARIO}"
OUTPUT_RAW = f"{OUTPUT}/{RUN}/graphics/demand/"

In [ ]:
n = pypsa.Network(INPUT)

### Analyse the first hours

Assumption: The network has to deliver export hydrogen from hour 1 onwards. Does this drive up storage needs etc.?

Hydrogen demand: 11 GW every hour

In [ ]:
n.loads_t.p_set.loc[:, "H2 export load"]

In [ ]:
# Electricity demand of all electrolysers


Storage (stats)

Note: e/p ratio is not fixed, p is independent in a "store" component.

In [ ]:
# Optimised storage energy capacity
n.stores[n.stores.carrier == "H2"].e_nom_opt.sum() /1e3 # GWh

Storage (temporal)

In [ ]:
# State of charge (e) and power (p) of storage
n.stores_t.e.loc[:, n.stores[n.stores.carrier == "H2"].index].sum(axis=1).iloc[:100].plot(figsize=(11, 4), grid=True, label="store e")
n.stores_t.p.loc[:, n.stores[n.stores.carrier == "H2"].index].sum(axis=1).iloc[:100].plot(figsize=(11, 4), grid=True, label="store p")
n.generators_t.p.loc[:, n.generators[n.generators.bus.isin(n.buses[n.buses.carrier == "AC"].index)].index].sum(axis=1).iloc[:100].plot(figsize=(11, 4), grid=True, label="power")
n.links_t.p0.loc[:, n.links[n.links.carrier=="H2 Electrolysis"].index].sum(axis=1).iloc[:100].plot(label="electrolysis demand")
plt.legend()
plt.grid()

### Outcome

Storage Starts with 19 GWh, ends with 28 GWh. The full storage capacity is used throughout the year, not just in the first hour and then only marginally. Hence the storage capacity is not driven by the first hour.

### Export locations

In [ ]:
# Electricity demand of all electrolysers
n.links_t.p0.loc[:, n.links[n.links.carrier=="H2"].index].sum(axis=0)